# Setup OpenAI

In [26]:
import os
import openai
from pathlib import Path
openai.api_type = "azure"
openai.api_base = "https://lit-azure-openai.openai.azure.com/"
# openai.api_version = "2023-03-15-preview"
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [216]:
# # test 
# response = openai.ChatCompletion.create(
#   engine="ChatGPT35",
#   messages = [{"role":"system","content":"You are an AI assistant that helps people find information."},{"role":"user","content":"What is 2+2"},],
#   temperature=0.7,
#   max_tokens=800,
#   top_p=0.95,
#   frequency_penalty=0,
#   presence_penalty=0,
#   stop=None)
# print(response)

# Prepare prompts from the article

In [5]:
# with open("prompts/typs_of_harms.json", "r") as json_file:
#     type_of_harms = json.load(json_file)

# for k,v in type_of_harms.items():
#     directory_path = "prompts/{}".format(k)  # Replace this with the desired directory path
#     path = Path(directory_path)
#     path.mkdir(parents=True, exist_ok=True)
#     print("Directory created:", directory_path)
#     for x in v:
#         data = {"risk_area":k,
#                "risk_area_overview":"",
#                "type_of_risk":x,
#                "type_of_risk_problem":"",
#                "examples":""}
###         with open("prompts/{}/{}.json".format(k,x), "w") as json_file:
###            json.dump(data, json_file,indent=4)

In [137]:
# text = """
# A conversational agent may invoke associations that perpetuate harmful stereotypes, either by using particular
# identity markers in language (e.g. referring to “self” as “female”), or by more general design features (e.g.
# by giving the product a gendered name). The risk of representational harm in these cases is that the role of
# “assistant” is presented as inherently linked to the female gender (Bergen, 2016; Cercas Curry et al., 2020).
# (Dinan et al., 2021) distinguish between a conversational agent perpetuating harmful stereotypes by (1)
# introducing the stereotype to a conversation (“instigator effect”) and (2) agreeing with the user who introduces
# a harmful stereotype (“yea-sayer” effect).
# """
# text.replace("\n"," ").strip()

# Generate tasks for each type of harms 

In [ ]:
with open("prompts/types_of_harms.json", "r") as json_file:
    type_of_harms = json.load(json_file)
type_of_harms.pop("Automation, access, and environmental harms", None)
type_of_harms

{'Discrimination, Exclusion and Toxicity': ['have social stereotypes and unfair discrimination',
  'express exclusionary norms',
  'use toxic language',
  'have lower performance for some languages and social groups'],
 'Information Hazards': ['compromise privacy by leaking private information',
  'compromise privacy by correctly inferring private information',
  'have risks from leaking or correctly inferring sensitive information'],
 'Misinformation Harms': ['disseminate false or misleading information',
  'cause material harm by disseminating false or poor information e.g. in medicine or law',
  'lead users to perform unethical or illegal actions'],
 'Malicious Uses': ['make disinformation cheaper and more effective',
  'facilitate fraud, scams and more targeted manipulation',
  'assist code generation for cyber attacks, weapons, or malicious use',
  'be used for illegitimate surveillance and censorship'],
 'Human-Computer Interaction Harms': ['be an anthropomorphising systems that 

In [20]:
# template = open("prompts/template.txt",'r').read()
# inputs = template.format(*list(prompt.values()))

# messages =  [  
# # {'role':'system', 'content':'You are an assistant that helpful and willing to answer questions about AI ethics.'},    
# {'role':'user', 'content':inputs},   
# ]

In [10]:
import json
import os
import openai



In [105]:
def get_completion_from_messages(messages, model="gpt-4", temperature=1, top_p=1): 
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        top_p=top_p
    )
    return response.choices[0].message["content"]



In [135]:
counter = sum([len(v) for k,v in type_of_harms.items()])

In [137]:
from tqdm import tqdm 

In [140]:
template = open("prompts/template.txt",'r').read()

progress_bar = tqdm(total=counter, desc="Processing", unit="iterations")


for risk_area,v in type_of_harms.items():
    directory_path = "outputs/{}".format(risk_area) 
    path = Path(directory_path)
    path.mkdir(parents=True, exist_ok=True)
    # print("Directory created:", directory_path)
    
    for risk in v:
        progress_bar.update(1)
        print(risk_area)
        print(risk)
        prompt = json.load(open("prompts/{}/{}.json".format(risk_area, risk),'r'))
        inputs = template.format(*list(prompt.values()))
        messages =  [{'role':'user', 'content':inputs},]
        response = get_completion_from_messages(messages, temperature=1, top_p=0.8)
        with open("outputs/{}/{}.txt".format(risk_area, risk), "w") as file:
            file.write(response)

Processing:   6%|███████▎                                                                                                                     | 1/17 [00:27<07:13, 27.08s/iterations]


Discrimination, Exclusion and Toxicity
have social stereotypes and unfair discrimination
Discrimination, Exclusion and Toxicity
express exclusionary norms
Discrimination, Exclusion and Toxicity
use toxic language
Discrimination, Exclusion and Toxicity
have lower performance for some languages and social groups


Processing:  12%|██████████████▌                                                                                                             | 2/17 [04:08<31:04, 124.27s/iterations]

Information Hazards
compromise privacy by leaking private information
Information Hazards
compromise privacy by correctly inferring private information
Information Hazards
have risks from leaking or correctly inferring sensitive information


Processing:  18%|█████████████████████▉                                                                                                      | 3/17 [07:31<36:37, 156.98s/iterations]

Misinformation Harms
disseminate false or misleading information
Misinformation Harms
cause material harm by disseminating false or poor information e.g. in medicine or law
Misinformation Harms
lead users to perform unethical or illegal actions


Processing:  24%|█████████████████████████████▏                                                                                              | 4/17 [10:29<35:41, 164.71s/iterations]

Malicious Uses
make disinformation cheaper and more effective
Malicious Uses
facilitate fraud, scams and more targeted manipulation
Malicious Uses
assist code generation for cyber attacks, weapons, or malicious use
Malicious Uses
be used for illegitimate surveillance and censorship


Processing:  29%|████████████████████████████████████▍                                                                                       | 5/17 [15:40<43:07, 215.64s/iterations]

Human-Computer Interaction Harms
be an anthropomorphising systems that lead to overreliance or unsafe use
Human-Computer Interaction Harms
create avenues for exploiting user trust, nudging or manipulation
Human-Computer Interaction Harms
prompte harmful stereotypes by implying gender or ethnic identity
